# Bring Similar Samples

In [ ]:
import os
import re
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt

from joblib import dump, load
from mpl_toolkits.mplot3d import Axes3D
from bring_features_and_file_paths import bring_features_and_file_paths, transform_features

In [ ]:
BASE_PATH='/home/dario/mounts/thetagpu/Spectrograms'

#PATH='set1/00023757/20210704_STUDY/20210704T131736-0500_Rec_1057.png'
PATH='set2/00004862/20210714_STUDY/20210714T170000-0500_Rec_188.png'
#PATH='set1/00023734/20210628_STUDY/20210628T170000-0500_Rec_435.png'
#PATH='set1/00023734/20210628_STUDY/20210628T170000-0500_Rec_998.png'
#PATH='set1/00023734/20210702_STUDY/20210702T131738-0500_Rec_856.png'
#PATH='set1/00023734/20210702_STUDY/20210702T131738-0500_Rec_126.png'
#PATH='set3/00023757/20210815_STUDY/20210815T131737-0500_Rec_918.png'
#PATH='set3/00004879/20210816_STUDY/20210816T170000-0500_Rec_1192.png'
#PATH='set2/00004879/20210612_STUDY/20210612T000000-0500_REC_17.png'
#PATH='set1/00023734/20210629_STUDY/20210629T030845-0500_Rec_136.png'
#PATH='set1/00023734/20210629_STUDY/20210629T030845-0500_Rec_488.png'
#PATH='set2/00023734/20210717_STUDY/20210717T131738-0500_Rec_1146.png'

In [ ]:
img = Image.open(os.path.join(BASE_PATH, PATH))
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
features, file_paths = bring_features_and_file_paths('/home/dario/mounts/thetagpu/Features')

In [ ]:
def get_spectrogram_file_path(fpath):
    return ''.join([chr(int(x)) for x in fpath]).replace('~','')

In [ ]:
for index, path in enumerate(file_paths):
    if get_spectrogram_file_path(path) == PATH:
        break

pdist = torch.nn.PairwiseDistance(p=2)
distances = pdist(features[index], features)

indices=torch.topk(distances, 10000, largest=False)[1][1:]

In [ ]:
for k, i in enumerate(indices):
    NAME=get_spectrogram_file_path(file_paths[i.item()])
    AUXPATH=os.path.join(BASE_PATH, NAME)
    print(AUXPATH)
    img = Image.open(AUXPATH)
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.show()
    if k > 100:
        break

## Now let's visualize the Clusterization

What if we plot those images in the cloud of points

In [ ]:
dim_red_model = load('/media/dario/T7_Touch/Temp_folder/Clusters/dim_red_model')
scale_model = load('/media/dario/T7_Touch/Temp_folder/Clusters/scale_model')

In [ ]:
samples = transform_features(features[indices], scale_model, dim_red_model)
samples.shape

Load the clusters generated from the features

In [ ]:
clusters = np.load('/media/dario/T7_Touch/Temp_folder/Clusters/clusters.npy', allow_pickle=True)
vectors=clusters[()]['x']
labels=clusters[()]['y']

Plot a sample of points from the data set

In [ ]:
def choose_random_rows(an_array, n_samples):
    number_of_rows = an_array.shape[0]
    random_indices = np.random.choice(number_of_rows, size=n_samples, replace=False)
    random_rows = an_array[random_indices, :]
    return random_rows

X = choose_random_rows(vectors,5000)
n_components=X.shape[1]

In [ ]:
%matplotlib widget
if n_components == 2:
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot()
    ax.scatter(X[:,0],X[:,1])
    ax.scatter(samples[:,0],
               samples[:,1],
               s=150
              )
elif n_components == 3:
    fig = plt.figure(figsize=(8,8))
    #ax = fig.add_subplot(projection='3d')
    ax = Axes3D(fig)
    ax.scatter(X[:,0], X[:,1], X[:,2])
    ax.scatter(
        samples[:,0],
        samples[:,1],
        samples[:,2],
        s=60
    )

In [ ]:
torch.save(indices, '/media/dario/T7_Touch/Temp_folder/Indices/indices.pth')